In [2]:
# 1 imports
import wfdb, matplotlib.pyplot as plt, numpy as np
from scipy.signal import butter, filtfilt #FOR preprocessing



#print(wfdb.__version__)
# 2 load one record
record = wfdb.rdrecord(r"C:\Users\oussk\OneDrive\Desktop\02 y 02 s\Advanced Logic Design\ecg-project\data\raw\100")
ann= wfdb.rdann(r"C:\Users\oussk\OneDrive\Desktop\02 y 02 s\Advanced Logic Design\ecg-project\data\raw\100", 'atr')

length=2 #PERSONAL CHOICE TO BE JUSTIFIED
method="minmax" #can be modified later

signal = record.p_signal[:,0] #channel 0
fs = record.fs               #sampling rate
segment = signal[0:fs*length]

def bandpass_filter(signal, fs, low=0.5, high=40.0, order=4):
    """Apply band-pass Butterworth filter to ECG signal."""
    nyq = 0.5 * fs
    low_cut = low / nyq
    high_cut = high / nyq
    b, a = butter(order, [low_cut, high_cut], btype='band')
    return filtfilt(b, a, signal)

def preprocess_segment(segment, fs, method):
    """Filter + normalize one ECG segment."""
    # Step 1: Band-pass filter
    filtered = bandpass_filter(segment, fs)

    # Step 2: Normalization
    if method == "zscore":
        return (filtered - np.mean(filtered)) / (np.std(filtered) + 1e-8)
    elif method == "minmax":
        return 2 * (filtered - np.min(filtered)) / (np.max(filtered) - np.min(filtered) + 1e-8) - 1
    else:
        return filtered

def extract_segments(record, ann, fs, win_sec,method):
    """
    Cut the ECG into fixed-length windows and assign binary labels.
    Label = "abnormal" if any non-N beat is inside the window.
    """
    win_size = int(win_sec * fs)
    X, y = [], []

    signal = record.p_signal[:,0]   # use lead MLII
    n_samples = len(signal)

    # loop with stride = window size (no overlap)
    for start in range(0, n_samples - win_size, win_size):
        end = start + win_size
        seg = signal[start:end]
        seg = preprocess_segment(seg, fs, method)

        # find annotation symbols within this window
        mask = (ann.sample >= start) & (ann.sample < end)
        ann_in_window = np.array(ann.symbol)[mask]  # convert to array before indexing

        # define binary label: 0=Normal, 1=Abnormal
        if any(sym != 'N' for sym in ann_in_window):
            label = 1 #ABNORMAL
        else:
            label = 0 #NORMAL

        X.append(seg)
        y.append(label)

    return np.array(X), np.array(y)

arrX,arrY=extract_segments(record,ann,fs,length,method)
print(arrX.shape) #(902 windows of 720 length each)
print(arrY)
print("Labels distribution:", np.bincount(arrY))  # counts of normal vs abnormal


(902, 720)
[1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0